# Tech Co-Movement Analysis

**Research Question:** Do MAG7 stocks move together as a sector? Does sector-wide sentiment relate to individual stock movements?

**Exploration Plan:**
- Calculate sector-level sentiment and returns (daily averages across all 7 tickers)
- Look at relationships between:
  - Individual ticker sentiment and its own price movement
  - Sector-wide sentiment and individual ticker movements
  - Patterns in who moves with the group vs independently

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the joined dataset
df = pd.read_csv('../data/processed/gdelt_ohlcv_join.csv', parse_dates=['seendate', 'article_date', 'price_date'])

print(f"Loaded {len(df):,} article-price pairs")
print(f"Date range: {df['article_date'].min()} to {df['article_date'].max()}")
print(f"\nTickers: {sorted(df['ticker'].unique())}")

## Next Steps (To Do)

1. Calculate daily returns for each ticker: `(next_close - prev_close) / prev_close`
2. Aggregate by `price_date` to get:
   - Average sentiment per ticker per day
   - Sector-wide average sentiment per day
   - Sector-wide average return per day
3. Explore patterns:
   - How correlated are ticker movements?
   - Do tickers with similar sentiment move similarly?
   - When one ticker has strong sentiment, do others follow?
4. Visualizations:
   - Time series: sector sentiment vs individual ticker returns
   - Scatter: sector sentiment vs individual returns (colored by ticker)
   - Correlation matrix: price movements across tickers

---

## Notes

- **Data Source Note:** yfinance aggregates from multiple sources (Yahoo Finance is a news aggregator, not primary source). This means price data comes from exchange feeds, but news coverage patterns may reflect Yahoo's editorial choices.
- **Assumption:** Articles on day t align with prices on day t+1 (next trading day, weekends/holidays handled via NYSE calendar)
- **Limitation:** Sentiment scores are word-bank based (not ML), may miss context/sarcasm